## Contexto

Em uma transportadora, temos um problema onde temos produtos com características de nome, tamanho (em metros cúbicos) e valor, assim como, temos um caminhão com um limite de tamanho. Precisamos levar o máximo de produtos que conseguirmos, porém, é necessário priorizar o valor dos produtos, ou seja, os produtos mais caros que consequentemente darão mais lucro deverão estar no caminhão sem ultrapassar o valor máximo de tamanho.

Para isso, é necessário criar um algoritmo genético onde a solução tem como objetivo **maximizar o valor de lucro** nesse transporte para a transportadora.

### Classe produtos

Primeiramente, iremos declarar a classe produto.

In [1]:
class Produto():
    def __init__(self, nome, espaco, valor):
        self.nome = nome
        self.espaco = espaco
        self.valor = valor

In [2]:
lista_produtos: list[Produto] = []

lista_produtos.append(Produto("Geladeira Dako", 0.751, 999.90))
lista_produtos.append(Produto("Iphone 6", 0.0000899, 2911.12))
lista_produtos.append(Produto("TV 55' ", 0.400, 4346.99))
lista_produtos.append(Produto("TV 50' ", 0.290, 3999.90))
lista_produtos.append(Produto("TV 42' ", 0.200, 2999.00))
lista_produtos.append(Produto("Notebook Dell", 0.00350, 2499.90))
lista_produtos.append(Produto("Ventilador Panasonic", 0.496, 199.90))
lista_produtos.append(Produto("Microondas Electrolux", 0.0424, 308.66))
lista_produtos.append(Produto("Microondas LG", 0.0544, 429.90))
lista_produtos.append(Produto("Microondas Panasonic", 0.0319, 299.29))
lista_produtos.append(Produto("Geladeira Brastemp", 0.635, 849.00))
lista_produtos.append(Produto("Geladeira Consul", 0.870, 1199.89))
lista_produtos.append(Produto("Notebook Lenovo", 0.498, 1999.90))
lista_produtos.append(Produto("Notebook Asus", 0.527, 3999.00))

[p.nome for p in lista_produtos]

['Geladeira Dako',
 'Iphone 6',
 "TV 55' ",
 "TV 50' ",
 "TV 42' ",
 'Notebook Dell',
 'Ventilador Panasonic',
 'Microondas Electrolux',
 'Microondas LG',
 'Microondas Panasonic',
 'Geladeira Brastemp',
 'Geladeira Consul',
 'Notebook Lenovo',
 'Notebook Asus']

### Classe indivíduo

A primeira fase de um algoritmo genético é **gerar a população inicial**. A população é caracterizada por um **conjunto de indivíduos**.

geracao=0 significa que quando um indivíduo for criado, ele inicialmente não evoluiu nada.

Iremos aleatorizar o cromossomo (solução do indivíduo) na inicialização da classe como 0 ou 1, pois existem 2 tipos de estados de um produto, *irá levar* ou *não irá levar* o produto no caminhão.

Para o nosso caso, a *nota_avaliacao* será o somatório dos valores (R$) da carga.

In [ ]:
from random import random

class Individuo():
    def __init__(self, espacos, valores, limite_espacos, geracao=0):
        self.espacos = espacos # Os espaços de todos os produtos que podem ser carregados (em m³)
        self.valores = valores # Valores em reais
        self.limite_espacos = limite_espacos # Limite do espaço (caminhão)
        self.nota_avaliacao = 0 # Cada indivíduo terá uma nota que definirá se ele é bom ou ruim comparado aos outros (R$)
        self.espaco_usado = 0
        self.geracao = geracao # Geração atual do indivíduo
        self.cromossomo = [] # Solução do indivíduo, cada um dos elementos do array é chamado de Gene
        
        # Inicializando a solução aleatóriamente
        for i in range(len(espacos)):
            if random() < 0.5: # 50% de chance de inicializar com zero ou um
                self.cromossomo.append('0')
            else:
                self.cromossomo.append('1')
    
    # Função de avaliação/aptidão (Fitness)
    def avaliacao(self):
        nota = 0
        soma_espacos = 0
        
        # Soma os espaços e os valores dos produtos
        for idx, i in enumerate(self.cromossomo):
            if i == "1":
                soma_espacos += self.espacos[idx]
                nota += self.valores[idx]
                
        # Caso a nota for maior que o limite, rebaixa a nota, pois não é uma boa solução
        if soma_espacos > self.limite_espacos:
            nota = 1 # Por padrão é utilizado o valor 1 para rebaixar a nota em algoritmos genéticos

        self.nota_avaliacao = nota
        self.espaco_usado = soma_espacos

In [4]:
import pandas as pd

espacos = []
valores = []
nomes = []

limite = 3

for p in lista_produtos:
    espacos.append(p.espaco)
    valores.append(p.valor)
    nomes.append(p.nome)

In [5]:
df = pd.DataFrame({
    'Nome': nomes,
    'Espaço': espacos,
    'Valor':valores,
})

df.head()

,Nome,Espaço,Valor
0,Geladeira Dako,0.75100,999.90
1,Iphone 6,0.00009,2911.12
2,TV 55',0.40000,4346.99
3,TV 50',0.29000,3999.90
4,TV 42',0.20000,2999.00


**Declarando e exibindo uma classe indivíduo de exemplo**

Iremos criar o indivíduo com todos os espaços e valores disponíveis, o cromossomo (solução) será inicializada aleatóriamente e a geração será definida como 0 (zero) por padrão.

In [6]:
individuo_exemplo = Individuo(espacos, valores, limite)

In [7]:
print('Espaços = ', str(individuo_exemplo.espacos))
print('Valores = ', str(individuo_exemplo.valores))
print('Cromossomo (resultado) = ', str(individuo_exemplo.cromossomo))

print('\nComponentes da carga')
for i in range(len(lista_produtos)):
    if individuo_exemplo.cromossomo[i] == "1":
        print(f'Nome: {lista_produtos[i].nome} R$ {lista_produtos[i].valor}')

# Também é possível da forma abaixo:
# for idx, i in enumerate(individuo_exemplo.cromossomo):
#     if i == "1":
#         print(lista_produtos[idx].nome)

Espaços =  [0.751, 8.99e-05, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527]
Valores =  [999.9, 2911.12, 4346.99, 3999.9, 2999.0, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.0]
Cromossomo (resultado) =  ['0', '0', '1', '0', '0', '1', '0', '0', '0', '1', '1', '1', '1', '0']

Componentes da carga
Nome: TV 55'  R$ 4346.99
Nome: Notebook Dell R$ 2499.9
Nome: Microondas Panasonic R$ 299.29
Nome: Geladeira Brastemp R$ 849.0
Nome: Geladeira Consul R$ 1199.89
Nome: Notebook Lenovo R$ 1999.9


**Realizando a avaliação do indivíduo de exemplo**

In [12]:
# Realiza os cálculos para a avaliação
individuo_exemplo.avaliacao()

print(f'Nota (R$): {individuo_exemplo.nota_avaliacao} \nEspaço usado: {individuo_exemplo.espaco_usado:.4f} m³')

Nota (R$): 11194.97 
Espaço usado: 2.4384 m³
